# PART I

# Demographics in Mexico 2020

### INDEX
#### 1. Dependencies and set up  2. Clean data    3. Analysis: By region,By state, By municipality, By location

## 1. Dependencies and set up

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import seaborn as sns
import math
import statistics
import scipy.stats

In [2]:
#Download from: https://www.inegi.org.mx/programas/ccpv/2020/#Datos_abiertos

path_2020 = os.path.join('..', 'demographic info', '1_conjunto_de_datos_iter_00CSV20.csv')

In [3]:
demographic_mxn_2020= pd.read_csv(path_2020,low_memory=False,encoding='utf-8-sig')

#note: use utf-8-sig beacuse of spanish characters
#https://datascience.stackexchange.com/questions/12343/pandas-read-csv-with-spanish-characters

## 2. Clean data

#### The dataset contains 232 columns and 3.2 millions rows which contain information of Mexico's demographic census 2020. The dataset includes information by  state, municipality and location; geolocation, by age, by gender, religion, marital status, migration, unemployment, health insutance, household and education

In [4]:
demographic_mxn_2020.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


##### 2.1 DROP COLUMNS

#### Columns that have aggregated data will not be included, 63 columns will be excluded

In [5]:

#drop columns that we will not use
simple_df=demographic_mxn_2020.drop(demographic_mxn_2020.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,52,53,54,55,56,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,140,141,142,187,188]],axis=1)


In [6]:
simple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 169 entries, ENTIDAD to TAMLOC
dtypes: int64(6), object(163)
memory usage: 252.3+ MB


####  2.2 CLEAN DATA

In [7]:
#clean data
clean_df = simple_df.replace("*", 0)
clean_df_II = clean_df.replace('N/D', 0)

In [8]:
#list(clean_df_II.columns)

#### 2.3 CLEAN DUPLICATES

In [9]:
#CLEAN for duplicates

#Method example:https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/
#df.drop(df[df['Age'] < 25].index, inplace = True)
# Filter all rows for which :

clean_df_II.drop(clean_df_II[clean_df_II["NOM_ENT"]== "Total nacional"].index,inplace=True)
clean_df_II.drop(clean_df_II[clean_df_II["NOM_LOC"]== "Total de la Entidad"].index,inplace=True)
clean_df_II.drop(clean_df_II[clean_df_II["NOM_LOC"]== "Localidades de una vivienda"].index,inplace=True)
clean_df_II.drop(clean_df_II[clean_df_II["NOM_LOC"]== "Localidades de dos viviendas"].index,inplace=True)
clean_df_II.drop(clean_df_II[clean_df_II["NOM_LOC"]== "Total del Municipio"].index,inplace=True)

#clean_df_II.head()


In [10]:
#clean_df_II["NOM_ENT"].unique()


#### 2.4 CONCATANATE TWO COLUMNS  MUNICIPALITY LEVEL

#### Concatanate name of state and municipality because there are a lot of municiplities that have the same name, therefore its harder to identify them. Example, Municipality San Miguel (state Queretaro), San Miguel (state Chiapas),etc


In [11]:
# method: https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-pandas-dataframe
#example:df["period"] = df["Year"].astype(str) + df["quarter"]

clean_df_II["NOM_ENT_MUN"]=clean_df_II["NOM_ENT"].astype(str)+ " , " +clean_df_II["NOM_MUN"].astype(str)
clean_df_II.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189432 entries, 7 to 195660
Columns: 170 entries, ENTIDAD to NOM_ENT_MUN
dtypes: int64(6), object(164)
memory usage: 247.1+ MB


In [12]:
num_municipality=clean_df_II["NOM_ENT_MUN"].unique()
len(num_municipality)

2467

#### 2.5 CONCAT LOCAL LEVEL

In [13]:
clean_df_II["NOM_ENT_MUN_LOC"]=clean_df_II["NOM_ENT_MUN"].astype(str)+ " , " +clean_df_II["NOM_LOC"].astype(str)

In [14]:
#clean_df_II["NOM_ENT_MUN_LOC"].unique()

In [15]:
clean_df_II["NOM_ENT_MUN_LOC"].count()

189432

In [16]:
#dictionary, not used

#categories_regiones={"Aguascalientes":"CENTRO NORTE","Baja California":"NORTE","Baja California Sur":"CENTRO NORTE",
#            "Campeche":"SUR","Chiapas":"SUR","Chihuahua":"NORTE","Ciudad de M√©xico":"CENTRO", "Coahuila de Zaragoza":"NORTE",
#            "Colima":"CENTRO NORTE","Durango":"CENTRO NORTE","Guanajuato":"CENTRO", "Guerrero":"CENTRO", "Hidalgo":"SUR",
#            "Jalisco":"CENTRO", "Michoac√°n de Ocampo":"CENTRO NORTE","Morelos":"CENTRO NORTE","M√©xico":"CENTRO",
#            "Nayarit":"CENTRO NORTE","Nuevo Le√≥n":"NORTE","Oaxaca":"SUR","Puebla":"CENTRO","Quer√©taro":"CENTRO",
#            "Quintana Roo":"SUR","San Luis Potos√≠":"CENTRO NORTE","Sinaloa":"CENTRO NORTE","Sonora":"NORTE",
#            "Tabasco":"SUR","Tamaulipas":"NORTE","Tlaxcala":"CENTRO","Veracruz de Ignacio de la Llave":"SUR",
#            "Yucat√°n":"SUR","Zacatecas":"CENTRO NORTE"}

#### 2.6 ADD REGION

####  There are 32 states. BANXICO ( Central Mexican Bank) divided the states by geography and economic indicators.

In [17]:
#Add name of region
#Method:https://www.youtube.com/watch?v=5hHW-g0uSUA

#import datframe regiones
path_regiones = os.path.join('..', 'demographic info', '2_regiones.csv')
regiones= pd.read_csv(path_regiones)

#merge
df_merge_regiones=clean_df_II.merge(regiones,on="NOM_ENT",how="left")
df_merge_regiones.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC,NOM_ENT_MUN,NOM_ENT_MUN_LOC,REGION
0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102°17'45.768"" W","21°52'47.362"" N",1878,863893,...,77719,53589,2995,5984,63661,595,13,"Aguascalientes , Aguascalientes","Aguascalientes , Aguascalientes , Aguascalientes",CENTRO NORTE
1,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102°22'24.710"" W","21°52'18.749"" N",1902,5,...,0,0,0,0,0,0,1,"Aguascalientes , Aguascalientes","Aguascalientes , Aguascalientes , Granja Adelita",CENTRO NORTE
2,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102°21'25.639"" W","21°53'01.522"" N",1861,41,...,2,1,0,1,6,0,1,"Aguascalientes , Aguascalientes","Aguascalientes , Aguascalientes , Agua Azul",CENTRO NORTE
3,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],"102°21'26.261"" W","21°46'48.650"" N",1861,8,...,0,0,0,0,0,0,1,"Aguascalientes , Aguascalientes","Aguascalientes , Aguascalientes , Los Arbolito...",CENTRO NORTE
4,1,Aguascalientes,1,Aguascalientes,104,Ardillas de Abajo (Las Ardillas),"102°11'30.914"" W","21°56'42.243"" N",1989,1,...,0,0,0,0,0,0,1,"Aguascalientes , Aguascalientes","Aguascalientes , Aguascalientes , Ardillas de ...",CENTRO NORTE


#### 2.7 REARRANGE COLUMNS

In [18]:
df_semifinal=df_merge_regiones[["REGION","NOM_ENT_MUN",'ENTIDAD','NOM_ENT','MUN','NOM_MUN','LOC','NOM_LOC',"NOM_ENT_MUN_LOC",'LONGITUD','LATITUD',
 'ALTITUD','POBTOT','POBFEM','POBMAS','P_0A2','P_0A2_F','P_0A2_M','P_3A5','P_3A5_F','P_3A5_M',
 'P_6A11','P_6A11_F','P_6A11_M', 'P_8A14','P_8A14_F','P_8A14_M','P_12A14','P_12A14_F',
 'P_12A14_M','P_15A17', 'P_15A17_F', 'P_15A17_M', 'P_18A24', 'P_18A24_F','P_18A24_M','P_15A49_F', 'P_60YMAS', 'P_60YMAS_F',
 'P_60YMAS_M','PNACENT','PNACENT_F','PNACENT_M','PNACOE','PNACOE_F','PNACOE_M','P3A5_NOA','P3A5_NOA_F','P3A5_NOA_M','P6A11_NOA','P6A11_NOAF','P6A11_NOAM',
 'P12A14NOA','P12A14NOAF','P12A14NOAM','P15A17A','P15A17A_F','P15A17A_M','P18A24A','P18A24A_F','P18A24A_M','P8A14AN','P8A14AN_F','P8A14AN_M','P15YM_AN','P15YM_AN_F','P15YM_AN_M',
 'P15YM_SE', 'P15YM_SE_F', 'P15YM_SE_M','P15PRI_IN','P15PRI_INF','P15PRI_INM','P15PRI_CO','P15PRI_COF','P15PRI_COM',
 'P15SEC_IN','P15SEC_INF','P15SEC_INM','P15SEC_CO','P15SEC_COF','P15SEC_COM','P18YM_PB','P18YM_PB_F','P18YM_PB_M',
 'PEA','PEA_F','PEA_M','PE_INAC','PE_INAC_F','PE_INAC_M','POCUPADA','POCUPADA_F','POCUPADA_M','PDESOCUP',
 'PDESOCUP_F','PDESOCUP_M','PSINDER','PDER_SS','PDER_IMSS','PDER_ISTE','PDER_ISTEE','PAFIL_PDOM','PDER_SEGP',
 'PDER_IMSSB','PAFIL_IPRIV','PAFIL_OTRAI','P12YM_SOLT','P12YM_CASA','P12YM_SEPA','PCATOLICA','PRO_CRIEVA','POTRAS_REL',
 'PSIN_RELIG','TOTHOG','HOGJEF_F','HOGJEF_M','POBHOG','PHOGJEF_F','PHOGJEF_M','VIVTOT','TVIVHAB','TVIVPAR','VIVPAR_HAB',
 'VIVPARH_CV','TVIVPARHAB','VIVPAR_DES','VIVPAR_UT','OCUPVIVPAR','VPH_PISODT','VPH_PISOTI','VPH_1DOR','VPH_2YMASD',
 'VPH_1CUART','VPH_2CUART','VPH_3YMASC','VPH_C_ELEC','VPH_S_ELEC','VPH_AGUADV','VPH_AEASP','VPH_AGUAFV','VPH_TINACO',
 'VPH_CISTER','VPH_EXCSA','VPH_LETR','VPH_DRENAJ','VPH_NODREN','VPH_C_SERV','VPH_NDEAED','VPH_DSADMA','VPH_NDACMM',
 'VPH_SNBIEN','VPH_REFRI','VPH_LAVAD','VPH_HMICRO','VPH_AUTOM','VPH_MOTO','VPH_BICI','VPH_RADIO','VPH_TV','VPH_PC',
 'VPH_TELEF','VPH_CEL','VPH_INTER','VPH_STVP','VPH_SPMVPI','VPH_CVJ','VPH_SINRTV','VPH_SINLTC','VPH_SINCINT',
 'VPH_SINTIC','TAMLOC']]

#df_semifinal.head()

#### 2.8 change datatypes

In [19]:
#CHANGE DATATYPE

# columns that will not be change the data type
cols = df_semifinal.columns.drop(['REGION','NOM_ENT_MUN','ENTIDAD','NOM_ENT','MUN','NOM_MUN','LOC','NOM_LOC',"NOM_ENT_MUN_LOC",'LONGITUD','LATITUD','ALTITUD'])

#change to numeric instead of string
df_semifinal[cols] = df_semifinal[cols].apply(pd.to_numeric)


In [20]:
df_semifinal.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189432 entries, 0 to 189431
Data columns (total 172 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   REGION           189432 non-null  object
 1   NOM_ENT_MUN      189432 non-null  object
 2   ENTIDAD          189432 non-null  int64 
 3   NOM_ENT          189432 non-null  object
 4   MUN              189432 non-null  int64 
 5   NOM_MUN          189432 non-null  object
 6   LOC              189432 non-null  int64 
 7   NOM_LOC          189432 non-null  object
 8   NOM_ENT_MUN_LOC  189432 non-null  object
 9   LONGITUD         189432 non-null  object
 10  LATITUD          189432 non-null  object
 11  ALTITUD          189432 non-null  object
 12  POBTOT           189432 non-null  int64 
 13  POBFEM           189432 non-null  int64 
 14  POBMAS           189432 non-null  int64 
 15  P_0A2            189432 non-null  int64 
 16  P_0A2_F          189432 non-null  int64 
 17  P_0A2_M  

In [21]:
df_semifinal.to_csv("3_cleanDatabase.csv")

# 1990-2020

In [31]:
path_1990_2020 = os.path.join('..', 'demographic info', '0_demographics_INEGI_1990_2020.csv')

In [32]:
path_1990_2020= pd.read_csv(path_1990_2020,low_memory=False,encoding='utf-8-sig')

In [33]:
path_1990_2020.head()

,NOM_ENT,Grupo quinquenal de edad,AÑO,Total,Hombres,Mujeres
0,Aguascalientes,No especificado,1990,2245,1084,1161
1,Aguascalientes,No especificado,1995,1254,607,647
2,Aguascalientes,No especificado,2000,102,44,58
3,Aguascalientes,No especificado,2005,98,41,57
4,Aguascalientes,No especificado,2010,109,45,64


In [34]:
path_1990_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4246 entries, 0 to 4245
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   NOM_ENT                   4246 non-null   object
 1   Grupo quinquenal de edad  4246 non-null   object
 2   AÑO                       4246 non-null   int64 
 3   Total                     4246 non-null   int64 
 4   Hombres                   4246 non-null   int64 
 5   Mujeres                   4246 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 199.2+ KB


In [35]:

#add region
df_merge_regiones_1990_2020=path_1990_2020.merge(regiones,on="NOM_ENT",how="left")
df_merge_regiones_1990_2020.head()

,NOM_ENT,Grupo quinquenal de edad,AÑO,Total,Hombres,Mujeres,REGION
0,Aguascalientes,No especificado,1990,2245,1084,1161,CENTRO NORTE
1,Aguascalientes,No especificado,1995,1254,607,647,CENTRO NORTE
2,Aguascalientes,No especificado,2000,102,44,58,CENTRO NORTE
3,Aguascalientes,No especificado,2005,98,41,57,CENTRO NORTE
4,Aguascalientes,No especificado,2010,109,45,64,CENTRO NORTE


In [ ]:
#CHANGE DATATYPE

# columns that will not be change the data type
#cols = df_semifinal.columns.drop(['REGION','NOM_ENT_MUN','ENTIDAD','NOM_ENT','MUN','NOM_MUN','LOC','NOM_LOC',"NOM_ENT_MUN_LOC",'LONGITUD','LATITUD','ALTITUD'])

#change to numeric instead of string
#df_semifinal[cols] = df_semifinal[cols].apply(pd.to_numeric)


In [ ]:
df_merge_regiones_1990_2020.to_csv("4_timeline.csv")